# Evaluation of SSL methods

In this session we are going to implement the following evaluation modalities of SSL methods:
- kNN classifier
- linear probe
- MLP

1-2-3: dal più veloce e semplice da implementare a quello più complesso

We are going to use the ImageNet pretrained model of pytorch as pretrained model and test their performance of CIFAR10 (as we have already done in the CNN lab classes before in the course).

kNN:
Voglio estrarre tutte le features del target dataset con il modello pretrained (che è freezato). ((una volta ottenuto tutto abbiamo due split di training e di test)). di tutte le immagini di test andiamo a cercare i suoi k vicini del training. di questi vicini si prende la classe a cui l'elemento di test a cui è associato. Come si fa? Approccio vanilla, base: di quei k feature di training si va a contare qual è il numero maggiore di classi di quelle features (disegno)

Quindi la classe predetta y* dell'elemento * di test, sarà un counting delle classi (si fa una sorta di istogramma). A y* viene associata la classe (come approccio base) quella con più elementi. Conta qual è la classe più frequente dei miei vicini. L'unico parametro da scegliere è k.

Altro approccio: si fa una sorta di voting e non si prende il max dell'istogramma, si va a dare un peso alle classi in base alla loro frequenza. Questo perché, se si hanno dataset sbilanciati, prendere il max non sarebbe stato corretto.

Quindi i passaggi sono:

- si estraggono le features delle immagini di test e di training. Viene fatta una distanza di ogni feature di test da ogni feature di train, e poi si vanno a prendere i k più vicini e sulla base di questi k si associa una classe.

Linear Probe:
Si tiene la backbone fissata e ci si va ad appendere nel fc un linear layer che ha come numero di classi, quelle del target dataset (es 10 con cifar10) e si va ad addestrare solo il fc finale. modo più famoso per valutare metodi self supervised, ovvero solo appenderci un linear layer.

di solito si fa pretext con 95% di ImageNet e poi la evalution con linear prob con 5% di ImageNet. Questo è il benchmark più famoso. si usa il 5% come dato supervisionato. con il linear layer qui come uscita ho una softmax. PRO: veloce, meno veloce del knn classifier

più dati ho, più il gap tra supervised e unsupervised si avvicina

MLP: 
ci si mette una sequenza di linear e relu. è diverso dagli mlp usati durante il training. l'mlp qui (con cifar) ha 10 classi. l'mlp di training invece viene buttato via. si prende solo un encoder di una delle due. l'mlp rispetto alla linear probe ha il problema dell'"overfit", l'accuratezza dopo un po' scende, per questo si fa early stopping: dal training si prendono un po' di dati di valutazione e poi qui ci calcolo l'accuratezza, quando qui è massima prendo lo stato della rete e poi lo ricarico nell'mlp.

- ultima parte: finetuning. è bene scartala! è molto difficile come riuscire a confrontare questi metodi, si fa a finetunare i pesi della backbone, si muovono i suoi parametri. benchmark: 50-100 epoche e poi vare evaluation.

I metodi SSL possono essere implementati in infiniti modi, noi vogliamo vedere quello di classificazione.

FARE: implementare queste modalità, ma soprattutto implementare knn su CIFAR10. Implementare versione vanilla ovvero si va a prendere come classe predetta quella con il numero maggiore di vicini. classe più frequente nel vicinato. provare vari k.

REPORT:
commentare i paper, li presentiamo e diciamo le differenze. Parlare delle diverse 3 categorie di metodi che ci sono nel SSL + mettere un commento sui modelli + mettere parti di codice. (ES: come è fatto SimCLR, come è fatto il batch etc. la loss... + codice, commentare i passaggi principali, ad esempio che si è costruito un array dove nella prima pos ci sono i postive etc.)

non ci sono i risultati!!!

+  mettere la parte di evaluation, soprattutto la parte di knn, linear probe e mlp non ci interessano.
quindi knn prende in input le features e poi questo lavoro di confronto delle distanze delle feature di test con quelle di train

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models

In [ ]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])


batch_size = 64

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)


100%|██████████| 170498071/170498071 [00:02<00:00, 63416163.73it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


## Exercise 0

Create a linear probe on CIFAR10 using the pretrained model of torchvision, models.resnet18(pretrained=True).

Keep attention to set the model in evaluation mode with model.eval() during the training of the linear layer, since all the layers of the pretrained model should be used in eval mode (eg the batch norm/dropout layers, if present).

prendere cifar10 come dataset di test, prendere resnet18, (1-2 fare confronto)

## Exercise 1

Compare the linear evaluation with the MLP strategy.

Implement now a MLP on top of the model and train just this part.
Also in this case keep attention to set the model in evaluation mode with model.eval() during the training of the linear layer, since all the layers of the pretrained model should be used in eval mode (eg the batch norm/dropout layers, if present).

## Exercise 2

Implement the KNN classifier. Test serveral values of k.
